In [1]:
import gc
import os
import time
import sys
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from scipy import stats
from scipy.signal import hann
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.signal import convolve
from sklearn.svm import NuSVR, SVR
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.model_selection import KFold,StratifiedKFold, RepeatedKFold
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

warnings.filterwarnings("ignore")
import os
print(os.listdir("../input"))

['earthquake-data-overlapping4', 'LANL-Earthquake-Prediction']


In [2]:
train_X = pd.read_csv('../input/earthquake-data-overlapping4/train_X.csv')
train_y = pd.read_csv('../input/earthquake-data-overlapping4/train_y.csv')
test_X = pd.read_csv('../input/earthquake-data-overlapping4/test_X.csv')
train_X.head()

,mean,std,max,Rmean,Rstd,Rmax,Rmin,Imean,Istd,Imax,Imin,Rmean_last_5000,Rstd__last_5000,Rmax_last_5000,Rmin_last_5000,Rmean_last_15000,Rstd_last_15000,Rmax_last_15000,Rmin_last_15000,mean_change_abs,mean_change_rate,abs_max,abs_min,std_first_50000,std_last_50000,std_first_10000,std_last_10000,avg_first_50000,avg_last_50000,avg_first_10000,avg_last_10000,min_first_50000,min_last_50000,min_first_10000,min_last_10000,max_first_50000,max_last_50000,max_first_10000,max_last_10000,max_to_min_diff,...,q01_roll_std_5000,q05_roll_std_5000,q95_roll_std_5000,q99_roll_std_5000,av_change_abs_roll_std_5000,av_change_rate_roll_std_5000,abs_max_roll_std_5000,ave_roll_mean_5000,std_roll_mean_5000,max_roll_mean_5000,min_roll_mean_5000,q01_roll_mean_5000,q05_roll_mean_5000,q95_roll_mean_5000,q99_roll_mean_5000,av_change_abs_roll_mean_5000,av_change_rate_roll_mean_5000,abs_max_roll_mean_5000,ave_roll_std_10000,std_roll_std_10000,max_roll_std_10000,min_roll_std_10000,q01_roll_std_10000,q05_roll_std_10000,q95_roll_std_10000,q99_roll_std_10000,av_change_abs_roll_std_10000,av_change_rate_roll_std_10000,abs_max_roll_std_10000,ave_roll_mean_10000,std_roll_mean_10000,max_roll_mean_10000,min_roll_mean_10000,q01_roll_mean_10000,q05_roll_mean_10000,q95_roll_mean_10000,q99_roll_mean_10000,av_change_abs_roll_mean_10000,av_change_rate_roll_mean_10000,abs_max_roll_mean_10000
0,1.700578,0.633686,4.653960,2.564949,680.894175,255058.322797,-2408.516087,1.455192e-16,174.151534,3239.416833,-3239.416833,-0.654161,267.514872,3454.906692,-2408.516087,-0.830288,281.033541,3454.906692,-2408.516087,-1.709978e-05,74843.221402,4.653960,0.0,0.672039,0.604280,0.782109,0.626411,1.739996,1.632751,1.893847,1.670074,0.0,0.0,0.0,0.0,4.653960,3.433987,4.653960,3.295837,4.653960,...,0.529758,0.549506,0.693419,0.878553,-1.962850e-06,72228.057274,0.891800,1.696024,0.074840,2.068372,1.567778,1.574001,1.586487,1.794616,2.041310,-2.761095e-06,72228.057274,2.068372,0.620969,0.036920,0.783651,0.548856,0.553488,0.567385,0.668806,0.777202,-1.112140e-06,69766.232090,0.783651,1.692042,0.056575,1.893822,1.573180,1.578145,1.587725,1.763602,1.879227,-1.598609e-06,69766.232090,1.893822
1,1.687215,0.631170,5.204007,1.945910,675.975105,253066.608763,-2328.675199,1.576457e-16,172.455209,2535.094444,-2535.094444,-2.370291,269.970140,2423.243075,-2328.675199,-1.217297,280.304439,2423.243075,-2328.675199,-2.243170e-06,75068.853673,5.204007,0.0,0.620690,0.665901,0.646171,0.675836,1.716995,1.690943,1.693608,1.702647,0.0,0.0,0.0,0.0,4.043051,5.204007,3.713572,3.784190,5.204007,...,0.529758,0.547089,0.706186,0.903979,5.646436e-07,72504.404671,0.918148,1.686305,0.077480,1.991082,1.567778,1.574001,1.586487,1.816882,1.984399,4.394670e-07,72504.404671,1.991082,0.623624,0.051871,0.797961,0.548856,0.553488,0.567268,0.785174,0.794373,2.119372e-07,69994.498756,0.797961,1.685211,0.061058,1.831696,1.573180,1.578145,1.587725,1.808517,1.826149,6.460775e-08,69994.498756,1.831696
2,1.689548,0.645775,5.204007,2.079442,677.862304,253428.096553,-2562.230574,-1.576457e-16,176.639855,3228.149028,-3228.149028,1.211027,289.350010,3407.107638,-2562.230574,1.729464,289.262755,3407.107638,-2562.230574,-4.132616e-11,75056.126424,5.204007,0.0,0.603759,0.679278,0.549523,0.600103,1.691658,1.708584,1.726333,1.667097,0.0,0.0,0.0,0.0,3.970292,5.030438,3.258096,3.663562,5.204007,...,0.529758,0.554342,0.856148,0.903979,-8.374987e-08,72659.749225,0.918148,1.689388,0.081948,1.991082,1.567778,1.574001,1.586487,1.851968,1.984399,-5.674733e-07,72659.749225,1.991082,0.644311,0.063337,0.797961,0.549518,0.558957,0.583459,0.788207,0.794373,3.612787e-07,70241.279250,0.797961,1.688871,0.063925,1.831696,1.573180,1.578145,1.587725,1.808517,1.826149,-4.231743e-07,70241.279250,1.831696
3,1.681095,0.647735,5.204007,1.609438,675.114788,252142.605786,-2923.794506,-1.637090e-16,176.069659,3361.815243,-3361.815243,-2.515107,309.301760,2927.914965,-2923.794506,-1.065583,293.424315,2927.914965,-2923.794506,-6.108616e-06,75014.658586,5.204007,0.0,0.641665,0.616

對訓練資料進行尺度縮放的處理

In [3]:
scaler = StandardScaler()
scaler.fit(train_X)
scaled_train_X = pd.DataFrame(scaler.transform(train_X), columns=train_X.columns)
#scaled_train_X = train_X

In [4]:
scaled_test_X = pd.DataFrame(scaler.transform(test_X), columns=test_X.columns)
#scaled_test_X = test_X

In [5]:
scaled_train_X.head(), scaled_test_X.head()

(       mean           ...             abs_max_roll_mean_10000
 0  0.653634           ...                            0.226685
 1  0.440375           ...                           -0.045400
 2  0.477612           ...                           -0.045400
 3  0.342719           ...                           -0.045400
 4  0.622946           ...                           -0.045400
 
 [5 rows x 253 columns],
        mean           ...             abs_max_roll_mean_10000
 0 -0.344491           ...                           -0.313423
 1 -0.835153           ...                           -0.138089
 2  0.200758           ...                            0.483292
 3 -0.526375           ...                           -0.417852
 4 -1.059244           ...                           -0.044151
 
 [5 rows x 253 columns])

In [6]:
scaled_train_X.shape, scaled_test_X.shape

((16761, 253), (2624, 253))

In [7]:
train_X_mean = np.mean(scaled_train_X, axis=0)
train_X_mean.head()

mean     1.466831e-15
std      7.358564e-16
max     -2.618009e-16
Rmean    4.673621e-17
Rstd    -9.704731e-16
dtype: float64

In [8]:
test_X_mean = np.mean(scaled_test_X, axis=0)
test_X_mean.head()

mean    -0.842562
std      0.284385
max      0.039534
Rmean   -0.080621
Rstd    -0.688147
dtype: float64

In [10]:
c = 0
for _, i in (train_X_mean - test_X_mean).abs().sort_values().iteritems():
    print(i, _)
    if _ == 'std_roll_std_50':
        print(c - 9)
    c += 1

0.0 q01
0.0 q001
0.0 abs_q01
0.0 count_big
0.0 min_last_50000
0.0 min_first_10000
0.0 min_last_10000
0.0 min_first_50000
0.0 abs_min
9.375776612797745e-05 av_change_abs_roll_std_10000
0.0008578532076590063 av_change_abs_roll_mean_10000
0.0012692648292819766 q95_roll_mean_50
0.0019303365173806303 av_change_abs_roll_mean_100
0.0024279128238885177 q99_roll_mean_1000
0.002532143874549431 q95_roll_mean_100
0.0033452644099660283 max_last_10000
0.0035651638731541905 av_change_abs_roll_std_10
0.004243453021396381 av_change_abs_roll_std_5
0.004369214626621779 mean_change_abs
0.0056125779592634175 max_first_10000
0.006762595248573897 av_change_abs_roll_std_1000
0.006863464433817255 av_change_abs_roll_mean_50
0.007403280795349504 q99_roll_std_10
0.0077433109837537 mean_change_rate
0.008295769251100605 av_change_abs_roll_std_500
0.008634430470347962 av_change_abs_roll_mean_5
0.008849343691339592 av_change_rate_roll_std_50
0.008901853165883033 av_change_rate_roll_mean_5
0.009079429053427148 av_chan

In [ ]:
features = (train_X_mean - test_X_mean).abs().sort_values()
features = features[features > 0]
drop_features = features.index[167:]

In [ ]:
train_X_less = scaled_train_X.drop(drop_features, axis=1)
test_X_less = scaled_test_X.drop(drop_features, axis=1)

# Cross Validation Function

In [ ]:
n_fold = 5
def mae_cv (model):
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=42).get_n_splits(train_X_less.values)
    mae = -cross_val_score (model, train_X_less.values, train_y, scoring="neg_mean_absolute_error",
                           verbose=0,
                           cv=folds)
    return mae

## Random Forest

In [ ]:
%%time
rf_model = RandomForestRegressor(n_estimators=120, n_jobs=-1, min_samples_leaf=1, 
                           max_features = "auto",max_depth=15, )
#score = mae_cv(rf_model)
#print("Random Forest score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
rf_model

## Cat Boost

In [ ]:
%%time
params = {'loss_function':'MAE',}
cat_model = CatBoostRegressor(iterations=1000,  eval_metric='MAE', verbose=False, **params)

#score = mae_cv(cat_model)
#print("Cat Boost score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
cat_model

## Elastic Net

In [ ]:
%%time
#ENet_model = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=0.9, random_state=3,max_iter=5000))
ENet_model = ElasticNet(alpha=0.0005, l1_ratio=0.9, random_state=3,max_iter=5000)
#score = mae_cv(ENet_model)
#print("Elastic Net score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
ENet_model

## Lasso

In [ ]:
%%time
lasso_model = Lasso(alpha =0.0005, random_state=1)
#score = mae_cv(lasso_model)
#print("Lasso score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
lasso_model

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [ ]:
%%time
averaged_models = AveragingModels(models = (rf_model,cat_model))
#score_rf_cat = mae_cv(averaged_models_rf_cat)
#print(" Averaged base models score_rf_cat: {:.4f} ({:.4f})\n".format(score_rf_cat.mean(), score_rf_cat.std()))

#averaged_models_cat_enet = AveragingModels(models = (cat_model, ENet_model))
#score_cat_enet = mae_cv(averaged_models_cat_enet)
#print(" Averaged base models score_cat_enet: {:.4f} ({:.4f})\n".format(score_cat_enet.mean(), score_cat_enet.std()))

#averaged_models_cat_lasso = AveragingModels(models = (cat_model,lasso_model))
#score_cat_lasso = mae_cv(averaged_models_cat_lasso)
#print(" Averaged base models score_cat_lasso: {:.4f} ({:.4f})\n".format(score_cat_lasso.mean(), score_cat_lasso.std()))

In [ ]:
averaged_models.fit (train_X_less.values, train_y)
averaged_train_predict = averaged_models.predict(train_X_less.values)
print(mean_absolute_error(train_y, averaged_train_predict))

In [ ]:
averaged_prediction = np.zeros(len(test_X_less))
averaged_prediction += averaged_models.predict(test_X_less.values)
averaged_prediction

# Submission

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv', index_col='seg_id')
submission.head()

In [ ]:
submission.time_to_failure = averaged_prediction
submission.to_csv('submission_average_167_features.csv',index=True)